In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import numpy as np

torch.__version__

'1.12.1'

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### Get data

In [3]:
import requests
import zipfile
from pathlib import Path

data_path = Path('data/')
image_path = data_path / 'pizza_steak_sushi'

if image_path.is_dir():
    print(f'{image_path} directory already exists')
else:
    print(f'{image_path} directory does not exists, creating one...')
    image_path.mkdir(parents=True, exist_ok=True)
    
    with open(data_path / 'pizza_steak_sushi.zip', 'wb') as f:
        print('downloading, dataset...')
        req = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
        f.write(req.content)
    print('dataset downloaded')
        
    with zipfile.ZipFile(data_path / 'pizza_steak_sushi.zip', 'r') as zip_ref:
        print('unzipping dataset...')
        zip_ref.extractall(image_path)
    print(f'dataset unzipped in {image_path}')

data/pizza_steak_sushi directory does not exists, creating one...
downloading, dataset...
dataset downloaded
unzipping dataset...
dataset unzipped in data/pizza_steak_sushi


In [4]:
SEED = 42
train_dir = image_path / 'train'
test_dir = image_path / 'test'

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [6]:
from torchvision import datasets, transforms

# Create simple transform
data_transform = transforms.Compose([ 
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Use ImageFolder to create dataset(s)
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

test_data = datasets.ImageFolder(root=test_dir, 
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )


In [7]:
class_names = train_data.classes
output_shape = len(class_names)

class_names, output_shape

(['pizza', 'steak', 'sushi'], 3)

In [9]:
from torch.utils.data import DataLoader
import os

BATCH_SIZE = 32

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=BATCH_SIZE,
                              num_workers=os.cpu_count(),
                              shuffle=True)

test_dataloader = DataLoader(dataset=test_data,
                             num_workers=os.cpu_count(),
                             batch_size=BATCH_SIZE)

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7fdea971ff10>,
 <torch.utils.data.dataloader.DataLoader at 0x7fdea971ea10>)

In [10]:
import torch

from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/
  
  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3, # how big is the square that's going over the image?
                    stride=1, # default
                    padding=0), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2) # default stride value is same as kernel_size
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from? 
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )
    
  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x
      # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion

In [11]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model
torch.manual_seed(42)
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB) 
                  hidden_units=10, 
                  output_shape=len(train_data.classes)).to(device)
model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [13]:
from typing import Tuple

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
    
    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train()
  
  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0
  
  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item() 

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch 
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

In [14]:
def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:
    
    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval() 
  
  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0
  
  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)
  
          # 1. Forward pass
          test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred_logits, y)
          test_loss += loss.item()
          
          # Calculate and accumulate accuracy
          test_pred_labels = test_pred_logits.argmax(dim=1)
          test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
          
  # Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

In [15]:
from typing import Dict, List

from tqdm.auto import tqdm

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]} 
    For example if training for epochs=2: 
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]} 
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }
  
  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)
      
      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results


In [16]:
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.
  
  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)
  
  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

In [17]:
# Set random seeds
torch.manual_seed(42) 
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 5

# Recreate an instance of TinyVGG
model_0 = TinyVGG(input_shape=3, # number of color channels (3 for RGB) 
                  hidden_units=10, 
                  output_shape=len(train_data.classes)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters(), lr=0.001)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Train model_0 
model_0_results = train(model=model_0, 
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn, 
                        epochs=NUM_EPOCHS,
                        device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

# Save the model
save_model(model=model_0,
           target_dir="models",
           model_name="05_going_modular_cell_mode_tinyvgg_model.pth")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1063 | train_acc: 0.3047 | test_loss: 1.0983 | test_acc: 0.3116
Epoch: 2 | train_loss: 1.0995 | train_acc: 0.3320 | test_loss: 1.0699 | test_acc: 0.5417
Epoch: 3 | train_loss: 1.0863 | train_acc: 0.4922 | test_loss: 1.0800 | test_acc: 0.5227
Epoch: 4 | train_loss: 1.0826 | train_acc: 0.4102 | test_loss: 1.0599 | test_acc: 0.5729
Epoch: 5 | train_loss: 1.0630 | train_acc: 0.4141 | test_loss: 1.0609 | test_acc: 0.5540
[INFO] Total training time: 7.729 seconds
[INFO] Saving model to: models/05_going_modular_cell_mode_tinyvgg_model.pth


### Convert cells to python files

In [18]:
type(data_transform)

torchvision.transforms.transforms.Compose

In [22]:
%%writefile services/data_setup.py

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

class SetupData():
    """Creates training and testing dataloaders and datasets.

    Keyword arguments:
    train_dir -- Path to train directory.
    test_dir -- Path to test directory
    data_transform -- torchvision transforms to perform on training and testing data.
    batch_size -- number of samples per batch in each dataloader.
    num_workers  -- an integer for number of worker per dataloader.

    Return:
    A tuple of train_dataloder, test_dataloader, class_names.
    """

    def __init__(self,
                 train_dir: str,
                 test_dir: str,
                 batch_size: int = 32,
                 num_workers: int = os.cpu_count()) -> None:
        self.train_dir = str(train_dir)
        self.test_dir = str(test_dir)
        self.batch_size = batch_size
        self.num_workers = num_workers

    def create_transforms(self, train=False, test=False):
        """create_transforms create transforms for given split

        Args:
            train (bool, optional): if transform is for training dataset mark as true. Defaults to False.
            test (bool, optional): if transform is for testing dataset mark as true. Defaults to False.

        Returns:
            data_transform: transform for given split
        """
        if train:
            data_transform = transforms.Compose([
                transforms.Resize((64, 64)),
                transforms.ToTensor()])
        elif test:
            data_transform = transforms.Compose([
                transforms.Resize((64, 64)),
                transforms.ToTensor()])

        return data_transform

    def create_datasets(self):
        """create_datasets creates datasets for train and test directories.

        Returns:
            train_data, test_data: tuple of datasets for splits
        """
        train_transform = self.create_transforms(train=True)
        test_transform = self.create_transforms(test=True)

        train_data = datasets.ImageFolder(root=self.train_dir,
                                          transform=train_transform)

        test_data = datasets.ImageFolder(root=self.test_dir,
                                         transform=test_transform)
        return train_data, test_data

    def create_dataloaders(self):
        """create_dataloaders create dataloaders for train and test datasets.

        Returns:
            train_dataloader, test_dataloader, class_names: tuple for given split as well as the class_names of the dataset.
        """
        train_data, test_data = self.create_datasets()

        class_names = train_data.classes

        train_dataloader = DataLoader(dataset=train_data,
                                      batch_size=self.batch_size,
                                      num_workers=self.num_workers,
                                      shuffle=True,
                                      pin_memory=True)

        test_dataloader = DataLoader(dataset=test_data,
                                     shuffle=False,
                                     batch_size=self.batch_size,
                                     num_workers=self.num_workers,
                                     pin_memory=True)

        return train_dataloader, test_dataloader, class_names

Overwriting services/data_setup.py


In [23]:
train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [25]:
from services.data_setup import SetupData

setup_data = SetupData(train_dir=train_dir,
                       test_dir=test_dir)

train_dataloader, test_dataloader, class_names = setup_data.create_dataloaders()
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x7fddf9e27bb0>,
 ['pizza', 'steak', 'sushi'])

In [26]:
%%writefile services/model.py

import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/
  
  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=hidden_units, 
                    kernel_size=3, # how big is the square that's going over the image?
                    stride=1, # default
                    padding=0), # options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units, 
                    out_channels=hidden_units,
                    kernel_size=3,
                    stride=1,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2) # default stride value is same as kernel_size
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from? 
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*13*13,
                    out_features=output_shape)
      )
    
  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x
      # return self.classifier(self.block_2(self.block_1(x))) # <- leverage the benefits of operator fusion

Writing services/model.py


In [31]:
%%writefile services/model_builder.py

from services.model import TinyVGG


class BuildModel():
    """BuildModel Builds model based on model class in model.py

    Arguments:
    device -- device where the model will we allocated (cuda-cpu)
    input_shape -- number of channels for image (3, 1)
    output_shape -- number of classes
    hidden_units -- number of hidden units per layer
    """

    def __init__(self, device: str,
                 input_shape: int,
                 output_shape,
                 hidden_units: int):
        self.device = device
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.hidden_units = hidden_units

    def build_model(self):
        """build_model builds model.

        Returns:
            model: returns model with given parameters
        """
        model = TinyVGG(input_shape=self.input_shape,
                        output_shape=self.output_shape,
                        hidden_units=self.hidden_units).to(self.device)
        return model

Overwriting services/model_builder.py


In [34]:
input_shape = 3
input_shape, output_shape

(3, 3)

In [36]:
from services.model_builder import BuildModel

build_model = BuildModel(device=device,
                   input_shape=input_shape,
                   output_shape=output_shape,
                   hidden_units=10)

model = build_model.build_model()
model

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [37]:
next(model.parameters()).device

device(type='cuda', index=0)

In [42]:
img_batch, label_batch = next(iter(test_dataloader))
img, lbl = img_batch[0].unsqueeze(0).to(device), label_batch[0]

img, lbl

(tensor([[[[0.0431, 0.2078, 0.3725,  ..., 0.0824, 0.1333, 0.1020],
           [0.2039, 0.2000, 0.2627,  ..., 0.0784, 0.1255, 0.1137],
           [0.0980, 0.1176, 0.1333,  ..., 0.1059, 0.1255, 0.1255],
           ...,
           [0.0196, 0.0196, 0.0235,  ..., 0.0902, 0.1059, 0.0980],
           [0.0196, 0.0235, 0.0235,  ..., 0.0863, 0.1020, 0.0941],
           [0.0196, 0.0196, 0.0235,  ..., 0.0980, 0.0980, 0.0863]],
 
          [[0.0196, 0.1255, 0.2471,  ..., 0.0235, 0.0353, 0.0235],
           [0.1333, 0.1216, 0.1608,  ..., 0.0157, 0.0275, 0.0235],
           [0.0588, 0.0667, 0.0784,  ..., 0.0314, 0.0235, 0.0235],
           ...,
           [0.0039, 0.0039, 0.0078,  ..., 0.0196, 0.0235, 0.0196],
           [0.0039, 0.0078, 0.0078,  ..., 0.0196, 0.0235, 0.0196],
           [0.0039, 0.0039, 0.0078,  ..., 0.0314, 0.0235, 0.0196]],
 
          [[0.0118, 0.0392, 0.0588,  ..., 0.0039, 0.0078, 0.0039],
           [0.0471, 0.0314, 0.0392,  ..., 0.0039, 0.0078, 0.0039],
           [0.0314, 0.03

In [ ]:
model.eval()
with torch.inference_mode():
    res_test = model(img)

res_test

tensor([[0.0066, 0.0172, 0.0194]], device='cuda:0')

In [ ]:
pred_prob = torch.softmax(res_test, dim=1)
pred_lbl = torch.argmax(pred_prob)

In [ ]:
class_names[pred_lbl]

'sushi'

In [ ]:
class_names[lbl]

'pizza'

In [84]:
%%writefile services/engine.py

from typing import Tuple, Dict, List
from tqdm.auto import tqdm
import torch

class TrainTestStep():
    """
  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").
  """
    def __init__(self,
                model: torch.nn.Module, 
                train_dataloader: torch.utils.data.DataLoader, 
                test_dataloader: torch.utils.data.DataLoader, 
                optimizer: torch.optim.Optimizer,
                loss_fn: torch.nn.Module,
                epochs: int,
                device: torch.device):
        self.model = model
        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.epochs = epochs
        self.device = device
    
    def train_step(self) -> Tuple[float, float]:
        """Trains a PyTorch model for a single epoch.

        Turns a target PyTorch model to training mode and then
        runs through all of the required training steps (forward
        pass, loss calculation, optimizer step).

        Returns:
          A tuple of training loss and training accuracy metrics. 
          In the form (train_loss, train_accuracy). For example:

          (0.1112, 0.8743)
      """
        self.model.train()
        train_loss, train_acc = 0
        for batch, (X, y) in enumerate(self.train_dataloader):
            y_pred = self.model(X)
            loss = self.loss_fn(y_pred, y)
            train_loss += loss.item()
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
            train_acc += (y_pred_class == y).sum().item()/len(y_pred)
            
        train_loss /= len(self.train_dataloader)
        train_acc /= len(self.train_dataloader)
        
        return train_loss, train_acc
    
    def test_step(self):
        """Tests a PyTorch model for a single epoch.

        Turns a target PyTorch model to "eval" mode and then performs
        a forward pass on a testing dataset.

        Returns:
         A tuple of testing loss and testing accuracy metrics.
         In the form (test_loss, test_accuracy). For example:

          (0.0223, 0.8985)
        """
        self.model.eval()
        test_loss, test_acc = 0
        with torch.inference_mode():
            for batch, (X_test, y_test) in enumerate(self.test_dataloader):
                y_test_logits = self.model(X_test)
                y_test_loss = self.loss_fn(y_test_logits, y_test)
                test_loss += y_test_loss.item()
                self.optimizer.zero_grad()
                y_test_loss.backward()
                self.optimizer.step()
                test_pred_labels = y_test_logits.argmax(dim=1)
                test_acc += ((test_pred_labels == y_test).sum().item()/len(test_pred_labels))
                
            test_loss /= len(self.test_dataloader)
            test_acc /= len(self.test_dataloader)
            
            return test_loss, test_acc
        
    def train_model(self):
        results = {
                "train_loss": [],
                "train_acc": [],
                "test_loss": [],
                "test_acc": []}
            
        for epoch in tqdm(range(self.epochs)):
            train_loss, train_acc = self.train_step()
            test_loss, test_acc = self.test_step()
                
            print(f"Epoch: {epoch+1} | "
                  f"train_loss: {train_loss:.4f} | "
                  f"train_acc: {train_acc:.4f} | "
                  f"test_loss: {test_loss:.4f} | "
                  f"test_acc: {test_acc:.4f}")
                
            results["train_loss"].append(train_loss)
            results["train_acc"].append(train_acc)
            results["test_loss"].append(test_loss)
            results["test_acc"].append(test_acc)
                
        return results

Overwriting services/engine.py


In [82]:
loss_fn, optimizer

(CrossEntropyLoss(),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     eps: 1e-08
     foreach: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ))

In [85]:
from services.engine import TrainTestStep

In [86]:
train_test_step = TrainTestStep(model=model_0,
                      train_dataloader=train_dataloader,
                      test_dataloader=test_dataloader,
                      optimizer=optimizer,
                      loss_fn=loss_fn,
                      epochs=5,
                      device=device)

In [87]:
results = train_test_step.train_model()

AttributeError: 'TrainTestStep' object has no attribute 'train_model'

In [90]:
%%writefile services/utils.py
from pathlib import Path
import torch

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.

  Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.
  
  Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)
  
  # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

  # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

Overwriting services/utils.py


In [91]:
from services.utils import save_model
save_model(model_0, target_dir='models', model_name='test.pth')

[INFO] Saving model to: models/test.pth


In [96]:
data_path = Path('./data/pizza_steak_sushi/')
train_dir = data_path / 'train'
test_dir = data_path / 'test'

train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

In [99]:
# %%writefile services/train.py

import torch
import torch.nn as nn
from timeit import default_timer as timer
from model_services.model_builder import BuildModel
from model_services.data_setup import SetupData
from model_services.utils import save_model
from model_services.engine import TrainTestStep
from pathlib import Path

data_path = Path('./data/pizza_steak_sushi/')
save_model_path = Path('./models/')
train_dir = data_path / 'train'
test_dir = data_path / 'test'
save_model_dir = save_model_path
model_name = 'test_model_v1.pth'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

setup_data = SetupData(train_dir=train_dir, test_dir=test_dir)
train_dataloader, test_dataloader, class_names = setup_data.create_dataloaders()

input_shape = 3
output_shape = len(class_names)
hidden_units = 10

build_model = BuildModel(device=device,
                         input_shape=input_shape,
                         output_shape=output_shape,
                         hidden_units=hidden_units)

model = build_model.build_model()

optimizer = torch.optim.Adam(params=model.parameters())
loss_fn = nn.CrossEntropyLoss()

epochs = 5

train_test_step = TrainTestStep(model=model,
                                train_dataloader=train_dataloader,
                                test_dataloader=test_dataloader,
                                optimizer=optimizer,
                                loss_fn=loss_fn,
                                epochs=epochs,
                                device=device)

results = train_test_step.train_model()

save_model(model=model,
           target_dir=save_model_path,
           model_name=model_name)

print(f'Model results : {results}')

ModuleNotFoundError: No module named 'model'

In [95]:
!python model_services/train.py

  0%|                                                     | 0/5 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/home/dev/projects/pytorch/My notebooks/Pytorch modular/model_services/train.py", line 60, in <module>
    run()
  File "/home/dev/projects/pytorch/My notebooks/Pytorch modular/model_services/train.py", line 50, in run
    results = train_test_step.train_model()
  File "/home/dev/projects/pytorch/My notebooks/Pytorch modular/model_services/engine.py", line 104, in train_model
    train_loss, train_acc = self.train_step()
  File "/home/dev/projects/pytorch/My notebooks/Pytorch modular/model_services/engine.py", line 49, in train_step
    train_loss, train_acc = 0
TypeError: cannot unpack non-iterable int object
